# Make Arrow Files

Process raw data to make Arrow files.

In [ ]:
library(ArchR)

In [3]:
set.seed(1)
addArchRThreads(threads = 32) 

Setting default number of Parallel threads to 32.



In [ ]:
addArchRGenome("hg38")

In [26]:
FRAG_BASE = "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/"
frag.files =  list.files(FRAG_BASE, pattern="*gz$")
sample.names = lapply(strsplit(frag.files, "_"), "[[", 1)
frag.files = paste(FRAG_BASE, frag.files, sep='')
names(frag.files) = sample.names

frag.files

D0 
 "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D0_S1_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                  D10 
"/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D10_S6_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                  D12 
"/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D12_S7_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                  D14 
"/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D14_S8_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                   D2 
 "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D2_S5_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                   D4 
 "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D4_S2_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                   D6 
 "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D6_S3_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                   D8 
 "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/D8_S4_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz" 
                                                                                                                                                                                   H1 
 "/oak/stanford/groups/akundaje/surag/projects/scATAC-reprog/singlecell/snap/fragment/coordsorted/H1_S1_L001_R1_001.merged.nodup.no_chrM_MT.tn5.withBarcode.coord.sorted.fragment.gz"

In [27]:
ArrowFiles <- createArrowFiles(
  inputFiles = frag.files,
  sampleNames = names(frag.files),
  filterTSS = 4, #Dont set this too high because you can always increase later
  filterFrags = 1000, 
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-2364bbcb3f4-Date-2020-04-24_Time-02-27-15.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2020-04-24 02:27:15 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-2364bbcb3f4-Date-2020-04-24_Time-02-27-15.log



In [30]:
ArrowFiles

[1] "D8.arrow"  "H1.arrow"  "D6.arrow"  "D4.arrow"  "D0.arrow"  "D2.arrow" 
[7] "D14.arrow" "D10.arrow" "D12.arrow"

In [49]:
# rds to tsv
for (x in c("D0",
"D2",
"D4",
"D6",
"D8",
"D10",
"D12",
"D14",
"H1")) {
    r = readRDS(sprintf("./QualityControl/%s/%s-Pre-Filter-Metadata.rds", x, x))
    write.table(r, sprintf("./QualityControl/%s/%s-Pre-Filter-Metadata.tsv", x, x), sep='\t', row.names=F, quote=F)
}

In [ ]:
doubScores <- addDoubletScores(
  input = ArrowFiles,
  k = 10, #Refers to how many cells near a "pseudo-doublet" to count.
  knnMethod = "UMAP", #Refers to the embedding to use for nearest neighbor search.
  LSIMethod = 1
)

ArchR logging to : ArchRLogs/ArchR-addDoubletScores-23645911eff5-Date-2020-04-24_Time-03-12-31.log
If there is an issue, please report to github with logFile!

2020-04-24 03:12:32 : Batch Execution w/ safelapply!, 0 mins elapsed.

2020-04-24 03:12:32 : D8 (1 of 9) :  Computing Doublet Statistics, 0 mins elapsed.

D8 (1 of 9) : UMAP Projection R^2 = 0.99751

D8 (1 of 9) : UMAP Projection R^2 = 0.99751

ggrastr is not available for rastr of points, continuing without rastr!

To install ggrastr try : devtools::install_github('VPetukhov/ggrastr')

ggrastr is not available for rastr of points, continuing without rastr!

To install ggrastr try : devtools::install_github('VPetukhov/ggrastr')

ggrastr is not available for rastr of points, continuing without rastr!

To install ggrastr try : devtools::install_github('VPetukhov/ggrastr')

2020-04-24 03:16:45 : H1 (2 of 9) :  Computing Doublet Statistics, 4.229 mins elapsed.

H1 (2 of 9) : UMAP Projection R^2 = 0.70366

H1 (2 of 9) : UMAP Projecti

In [45]:
# moved files manually
paste("/srv/scratch/surag/scATAC-reprog/arrow/", ArrowFiles, sep='')

[1] "/srv/scratch/surag/scATAC-reprog/arrow/D8.arrow" 
[2] "/srv/scratch/surag/scATAC-reprog/arrow/H1.arrow" 
[3] "/srv/scratch/surag/scATAC-reprog/arrow/D6.arrow" 
[4] "/srv/scratch/surag/scATAC-reprog/arrow/D4.arrow" 
[5] "/srv/scratch/surag/scATAC-reprog/arrow/D0.arrow" 
[6] "/srv/scratch/surag/scATAC-reprog/arrow/D2.arrow" 
[7] "/srv/scratch/surag/scATAC-reprog/arrow/D14.arrow"
[8] "/srv/scratch/surag/scATAC-reprog/arrow/D10.arrow"
[9] "/srv/scratch/surag/scATAC-reprog/arrow/D12.arrow"

In [ ]:
# Did not filter doublets since most cells seem to lie on continuum, making detection of doublets near impossible

In [5]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /users/surag/anaconda3/envs/r36_cran/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] ArchR_0.9.1                 magrittr_1.5               
 [3] rhdf5_2.30.1                Matrix_1.2-18              
 [5] data.table_1.12.8           SummarizedExperiment_1.16.1
 [7] DelayedArray_0.12.3         BiocParallel_1.20.1        
 [9] matrixStats_0.56.0        